In [30]:
import sys  
sys.path.insert(0, '../')

import wandb
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set(style='darkgrid')
%matplotlib inline

import numpy as np
import cupy as cp
import scipy
import model.functions as f
from chainer import serializers

class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

In [40]:
name = 'gaussian2_fc_ma_2000_0'

# check id
api = wandb.Api()
runs = api.runs("liuyuezhang/cerebellum")
for run in runs:
    if run.name == name:
        run_id = run.id
        config = Bunch(run.config)
        print(run.name, run.id)
        break

gaussian2_fc_ma_2000_0 1bfob7tw


In [41]:
from data.gaussian import get_gaussian

d = 500
n = 1000

data = get_gaussian(d=d, n=n, c=2, mu=0.5, sigma=1, seed=config.seed)
in_size = d
out_size = 2

X, Y = zip(*data)
X = np.array(X).T
Y = f.one_hot(Y, out_size=out_size, dtype=float).T
X.shape, Y.shape

((500, 1000), (2, 1000))

In [42]:
from model.cerebellum import Cerebellum
model = Cerebellum(in_size=in_size, out_size=out_size, args=config)

# load
for file in os.listdir('../wandb'):
    if run_id in file:
        serializers.load_npz('../wandb' + '/' + file + '/model.pkl', model)
        break

In [43]:
Z = model.project(X.T).array.T
W_ref = model.purkinje.W.array

Z.shape, W_ref.shape

((2000, 1000), (2, 2000))

In [47]:
W = Y @ Z.T @ np.linalg.inv(Z @ Z.T)
W.shape

(2, 2000)

In [52]:
u = W[0]
v = W_ref[0]
scipy.spatial.distance.cosine(u, v)

1.0080718594213747

In [45]:
calc_dim(Z)

344.72578309349484

In [38]:
def calc_dim(X):
    C = X @ X.T
    return cp.trace(C) ** 2 / cp.trace(C @ C)